# Hello World!

Here's an example notebook with some documentation on how to access CMIP data.

In [1]:
%matplotlib inline

import xarray as xr
import intake

# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util 

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [2]:
print('hello world!')

hello world!


## Demonstrate how to use `intake-esm`
[Intake-esm](https://intake-esm.readthedocs.io) is a data cataloging utility that facilitates access to CMIP data. It's pretty awesome.

An `intake-esm` collection object establishes a link to a database that contains file locations and associated metadata (i.e., which experiement, model, the come from). 

### Opening a collection
First step is to open the collection by pointing the collection definition file, which is a JSON file that conforms to the [ESM Collection Specification](https://github.com/NCAR/esm-collection-spec). 

The collection JSON files are stored locally in this repository for purposes of reproducibility---and because Cheyenne compute nodes don't have Internet access. 

The primary source for these files is the [intake-esm-datastore](https://github.com/NCAR/intake-esm-datastore) repository. Any changes made to these files should be pulled from that repo. For instance, the Pangeo cloud collection is available [here](https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json).

In [3]:
if util.is_ncar_host():
    col = intake.open_esm_datastore("../catalogs/glade-cmip6.json")
else:
    col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")
col

pangeo-cmip6-ESM Collection with 28660 entries:
	> 10 activity_id(s)

	> 23 institution_id(s)

	> 48 source_id(s)

	> 29 experiment_id(s)

	> 86 member_id(s)

	> 23 table_id(s)

	> 190 variable_id(s)

	> 7 grid_label(s)

	> 28660 zstore(s)

	> 59 dcpp_init_year(s)

`intake-esm` is build on top of [pandas](https://pandas.pydata.org/pandas-docs/stable). It is possible to view the `pandas.DataFrame` as follows.

In [4]:
col.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
0,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,Amon,pr,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...,NaN
1,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,Amon,prsn,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...,NaN
2,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,Amon,tas,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...,NaN
3,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,Amon,tasmax,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...,NaN
4,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,Amon,tasmin,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...,NaN


It is possible to interact with the `DataFrame`; for instance, we can see what the "attributes" of the datasets are by printing the columns.

In [5]:
col.df.columns

Index(['activity_id', 'institution_id', 'source_id', 'experiment_id',
       'member_id', 'table_id', 'variable_id', 'grid_label', 'zstore',
       'dcpp_init_year'],
      dtype='object')

### Search and discovery

#### Finding unique entries
Let's query the data to see what models ("source_id"), experiments ("experiment_id") and temporal frequencies ("table_id") are available.

In [6]:
import pprint 
uni_dict = col.unique(['source_id', 'experiment_id', 'table_id'])
pprint.pprint(uni_dict, compact=True)

{'experiment_id': {'count': 29,
                   'values': ['ssp370', 'esm-ssp585', '1pctCO2-bgc', 'hist-bgc',
                              '1pctCO2', 'abrupt-4xCO2', 'historical',
                              'piControl', 'amip', 'esm-hist', 'esm-piControl',
                              'hist-GHG', 'hist-aer', 'hist-nat', 'dcppA-assim',
                              'dcppA-hindcast', 'dcppC-hindcast-noAgung',
                              'dcppC-hindcast-noElChichon',
                              'dcppC-hindcast-noPinatubo', 'highresSST-present',
                              'control-1950', 'hist-1950', 'deforest-globe',
                              'esm-ssp585-ssp126Lu', 'omip1', 'lgm', 'ssp126',
                              'ssp245', 'ssp585']},
 'source_id': {'count': 48,
               'values': ['BCC-ESM1', 'BCC-CSM2-MR', 'CanESM5', 'CNRM-ESM2-1',
                          'UKESM1-0-LL', 'GISS-E2-1-G', 'CESM2', 'GFDL-ESM4',
                          'AWI-CM-1-1-MR', 'CAM

#### Searching for specific datasets

Let's find all the dissolved oxygen data at annual frequency from the ocean for the `historical` and `ssp585` experiments.

In [7]:
cat = col.search(experiment_id=['historical', 'ssp585'], table_id='Oyr', variable_id='o2', grid_label='gn')
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
1480,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1549,CMIP,CCCma,CanESM5,historical,r10i1p2f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1619,CMIP,CCCma,CanESM5,historical,r11i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r11i1...,NaN
1715,CMIP,CCCma,CanESM5,historical,r12i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r12i1...,NaN
1811,CMIP,CCCma,CanESM5,historical,r13i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r13i1...,NaN
1908,CMIP,CCCma,CanESM5,historical,r14i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r14i1...,NaN
2004,CMIP,CCCma,CanESM5,historical,r15i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r15i1...,NaN
2100,CMIP,CCCma,CanESM5,historical,r16i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r16i1...,NaN
2196,CMIP,CCCma,CanESM5,historical,r17i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r17i1...,NaN
2293,CMIP,CCCma,CanESM5,historical,r18i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r18i1...,NaN


It might be desirable to get more specific. For instance, we may want to select only the models that have *both* `historical` and `ssp585` data. We coud do this as follows.

In [8]:
models = set(uni_dict['source_id']['values']) # all the models

for experiment_id in ['historical', 'ssp585']:
    query = dict(experiment_id=experiment_id, table_id='Oyr', 
                 variable_id='o2', grid_label='gn')  
    cat = col.search(**query)
    models = models.intersection({model for model in cat.df.source_id.unique().tolist()})

# ensure the CESM2 models are not included (oxygen was erroneously submitted to the archive)
models = models - {'CESM2-WACCM', 'CESM2'}

models = list(models)
models

['CanESM5', 'MIROC-ES2L', 'IPSL-CM6A-LR']

In [9]:
cat = col.search(experiment_id=['historical', 'ssp585'], table_id='Oyr', 
                 variable_id='o2', grid_label='gn', source_id=models)
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
1480,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1549,CMIP,CCCma,CanESM5,historical,r10i1p2f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1619,CMIP,CCCma,CanESM5,historical,r11i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r11i1...,NaN
1715,CMIP,CCCma,CanESM5,historical,r12i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r12i1...,NaN
1811,CMIP,CCCma,CanESM5,historical,r13i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r13i1...,NaN
1908,CMIP,CCCma,CanESM5,historical,r14i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r14i1...,NaN
2004,CMIP,CCCma,CanESM5,historical,r15i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r15i1...,NaN
2100,CMIP,CCCma,CanESM5,historical,r16i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r16i1...,NaN
2196,CMIP,CCCma,CanESM5,historical,r17i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r17i1...,NaN
2293,CMIP,CCCma,CanESM5,historical,r18i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r18i1...,NaN


### Loading data

`intake-esm` enables loading data directly into an [xarray.Dataset](http://xarray.pydata.org/en/stable/api.html#dataset).

Note that data on the cloud are in 
[zarr](https://zarr.readthedocs.io/en/stable/) and data on 
[glade](https://www2.cisl.ucar.edu/resources/storage-and-file-systems/glade-file-spaces) are stored as 
[netCDF](https://www.unidata.ucar.edu/software/netcdf/) files. This is opaque to the user.

`intake-esm` has rules for aggegating datasets; these rules are defined in the collection-specification file.

In [10]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False}, 
                                cdf_kwargs={'chunks': {}, 'decode_times': False})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 6 group(s)


`dset_dict` is a dictionary of `xarray.Dataset`'s; its keys are constructed to refer to compatible groups.

In [11]:
dset_dict.keys()

dict_keys(['CMIP.CCCma.CanESM5.historical.Oyr.gn', 'CMIP.IPSL.IPSL-CM6A-LR.historical.Oyr.gn', 'CMIP.MIROC.MIROC-ES2L.historical.Oyr.gn', 'ScenarioMIP.CCCma.CanESM5.ssp585.Oyr.gn', 'ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp585.Oyr.gn', 'ScenarioMIP.MIROC.MIROC-ES2L.ssp585.Oyr.gn'])

We can access a particular dataset as follows.

In [12]:
dset_dict['CMIP.CCCma.CanESM5.historical.Oyr.gn']

<xarray.Dataset>
Dimensions:             (bnds: 2, i: 360, j: 291, lev: 45, member_id: 35, time: 165, vertices: 4)
Coordinates:
  * j                   (j) int32 0 1 2 3 4 5 6 ... 284 285 286 287 288 289 290
  * time                (time) float64 182.5 547.5 912.5 ... 5.968e+04 6.004e+04
  * i                   (i) int32 0 1 2 3 4 5 6 ... 353 354 355 356 357 358 359
  * lev                 (lev) float64 3.047 9.454 16.36 ... 5.375e+03 5.625e+03
  * member_id           (member_id) <U9 'r10i1p1f1' 'r10i1p2f1' ... 'r9i1p2f1'
Dimensions without coordinates: bnds, vertices
Data variables:
    lev_bnds            (lev, bnds) float64 dask.array<chunksize=(45, 2), meta=np.ndarray>
    latitude            (j, i) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
    time_bnds           (time, bnds) float64 dask.array<chunksize=(165, 2), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    vertices_longitude  (j, i, ver

### More advanced queries

As motivation for diving into more advanced manipulations with `intake-esm`, let's consider the task of getting access to grid information in the `Ofx` table_id.

In [13]:
cat_fx = col.search(experiment_id=['historical', 'ssp585'], source_id=models, table_id='Ofx', grid_label='gn')
cat_fx.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
1435,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Ofx,areacello,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1436,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Ofx,deptho,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1437,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Ofx,sftof,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1438,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Ofx,thkcello,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1535,CMIP,CCCma,CanESM5,historical,r10i1p2f1,Ofx,areacello,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1536,CMIP,CCCma,CanESM5,historical,r10i1p2f1,Ofx,deptho,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1537,CMIP,CCCma,CanESM5,historical,r10i1p2f1,Ofx,sftof,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1538,CMIP,CCCma,CanESM5,historical,r10i1p2f1,Ofx,thkcello,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1605,CMIP,CCCma,CanESM5,historical,r11i1p1f1,Ofx,areacello,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r11i1...,NaN
1606,CMIP,CCCma,CanESM5,historical,r11i1p1f1,Ofx,deptho,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r11i1...,NaN


This, however, comes with lots of redundant information.

Additionally, it may be necessary to do more targeted manipulations of the search. For instance, we've found a handful of corrupted files on `glade` and might need to work around loading these. 

As an illustration of this, in the code below, we specify a list of to queries (in this case one) to eliminate.

In [14]:
import numpy as np

# specify a list of queries to eliminate
corrupt_data = [dict(variable_id='areacello', source_id='IPSL-CM6A-LR',
                     experiment_id='historical', member_id='r2i1p1f1')
               ]


# copy the dataframe 
df = cat_fx.df.copy()

# eliminate data
for elim in corrupt_data:
    condition = np.ones(len(df), dtype=bool)
    for key, val in elim.items():
        condition = condition & (df[key] == val)
    df = df.loc[~condition]
df    

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
1435,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Ofx,areacello,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1436,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Ofx,deptho,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1437,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Ofx,sftof,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1438,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Ofx,thkcello,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1535,CMIP,CCCma,CanESM5,historical,r10i1p2f1,Ofx,areacello,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1536,CMIP,CCCma,CanESM5,historical,r10i1p2f1,Ofx,deptho,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1537,CMIP,CCCma,CanESM5,historical,r10i1p2f1,Ofx,sftof,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1538,CMIP,CCCma,CanESM5,historical,r10i1p2f1,Ofx,thkcello,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1605,CMIP,CCCma,CanESM5,historical,r11i1p1f1,Ofx,areacello,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r11i1...,NaN
1606,CMIP,CCCma,CanESM5,historical,r11i1p1f1,Ofx,deptho,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r11i1...,NaN


We then drop duplicates.

In [15]:
df.drop_duplicates(subset=['source_id', 'variable_id'], inplace=True)

Now, since we've only retained one ensemble member, we need to eliminate that column. If we omit this step, `intake-esm` will throw an error, complaining that different variables are present for each ensemble member. Setting the `member_id` column to NaN precludes attempts to join along the ensemble dimension.

After this final manipulation, we copy the `DataFrame` back to the collection object and procede with loading the data.

In [16]:
df['member_id'] = np.nan
cat_fx.df = df

In [17]:
fx_dsets = cat_fx.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 3 group(s)


In [18]:
fx_dsets.keys()

dict_keys(['CMIP.CCCma.CanESM5.historical.Ofx.gn', 'CMIP.IPSL.IPSL-CM6A-LR.historical.Ofx.gn', 'CMIP.MIROC.MIROC-ES2L.historical.Ofx.gn'])

In [19]:
for key, ds in fx_dsets.items():
    print(ds.data_vars)

Data variables:
    areacello           (j, i) float32 dask.array<chunksize=(291, 360), meta=np.ndarray>
    latitude            (j, i) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
    longitude           (j, i) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    vertices_longitude  (j, i, vertices) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    deptho              (j, i) float32 dask.array<chunksize=(291, 360), meta=np.ndarray>
    sftof               (j, i) float32 dask.array<chunksize=(291, 360), meta=np.ndarray>
    type                |S3 ...
    lev_bnds            (lev, bnds) float64 dask.array<chunksize=(45, 2), meta=np.ndarray>
    thkcello            (lev, j, i) float32 dask.array<chunksize=(45, 291, 360), meta=np.ndarray>
Data variables:
    area            (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    areacello

## Demonstrate how spin-up a dask cluster

If you expect to require Big Data capabilities, here's how you spin up a [dask](https://dask.org) cluster using [dask-jobqueue](https://dask-jobqueue.readthedocs.io/en/latest/).

The syntax is different if on an NCAR machine versus the cloud.

In [20]:
if util.is_ncar_host():
    from ncar_jobqueue import NCARCluster
    cluster = NCARCluster(project='UCGD0006')
    cluster.adapt(minimum_jobs=1, maximum_jobs=10)
else:
    from dask_kubernetes import KubeCluster
    cluster = KubeCluster()
    cluster.adapt(minimum=1, maximum=10)
cluster

In [21]:
from dask.distributed import Client
client = Client(cluster) # Connect this local process to remote workers
client

Client Scheduler: tcp://10.32.15.21:43305 Dashboard: /user/0000-0002-4605-8991/proxy/8787/status,Cluster Workers: 1 Cores: 2 Memory: 11.50 GB
